In [6]:
!pip install openai-whisper


In [7]:
import whisper
model = whisper.load_model("medium")


In [36]:
audio_path = "/content/contoh_audio.mp3"

result = model.transcribe(audio_path, language="indonesian")
print("Transkripsi:", result["text"])


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transkripsi:  Aku hari ini membeli bakso, soto, dan ketupat di toko Makmur Jaya dengan total Rp50.000


In [65]:
import re
from datetime import datetime, timedelta

MONTHS = {
    "januari": "01",
    "februari": "02",
    "maret": "03",
    "april": "04",
    "mei": "05",
    "juni": "06",
    "juli": "07",
    "agustus": "08",
    "september": "09",
    "oktober": "10",
    "november": "11",
    "desember": "12",
}

def parse_date(phrase):
    """Mengubah tanggal dalam berbagai format menjadi DD-MM-YYYY."""
    today = datetime.today()
    year = today.year

    patterns = [
        r"(\d{1,2})\s+(\w+)\s+(\d{4})",
        r"(\d{1,2})\s+(\w+)",
        r"(\d{1,2})\s+bulan\s+(\d{1,2})\s+tahun\s+(\d{4})",
        r"(\d{1,2})\s+bulan\s+(\d{1,2})\s+(\d{4})",
    ]

    for pattern in patterns:
        match = re.search(pattern, phrase, re.IGNORECASE)
        if match:
            if len(match.groups()) == 3 and pattern == patterns[0]:
                day, month_str, year = match.groups()
                month = MONTHS.get(month_str.lower(), "01")
                return f"{int(day):02d}-{month}-{year}"

            elif len(match.groups()) == 2 and pattern == patterns[1]:
                day, month_str = match.groups()
                month = MONTHS.get(month_str.lower(), "01")
                return f"{int(day):02d}-{month}-{year}"

            elif len(match.groups()) == 3 and pattern == patterns[2]:
                day, month, year = match.groups()
                return f"{int(day):02d}-{int(month):02d}-{year}"

    if "hari ini" in phrase:
        return today.strftime("%d-%m-%Y")
    elif "kemarin" in phrase:
        return (today - timedelta(days=1)).strftime("%d-%m-%Y")
    elif match := re.search(r"(\d+)\s*hari yang lalu", phrase):
        days_ago = int(match.group(1))
        return (today - timedelta(days=days_ago)).strftime("%d-%m-%Y")

    return None

def parse_total(total_str):
    total_str = total_str.lower().replace("rp", "").replace(".", "").replace(",", "")
    if "ribu" in total_str:
        total_str = total_str.replace("ribu", "").strip()
        return int(total_str) * 1000
    elif "juta" in total_str:
        total_str = total_str.replace("juta", "").strip()
        return int(total_str) * 1000000
    elif "miliar" in total_str:
        total_str = total_str.replace("miliar", "").strip()
        return int(total_str) * 1000000000
    elif "triliun" in total_str:
        total_str = total_str.replace("triliun", "").strip()
        return int(total_str) * 1000000000000
    return int(total_str)

def extract_transcribed_data(text):
    pattern = re.compile(
        r"(?:\w+)\s+(hari ini|kemarin|\d+\s*hari yang lalu|tanggal\s*\d{1,2}-\d{1,2}-\d{4}|\d{1,2}\s+\w+\s+\d{4}|\d{1,2}\s+\w+|\d{1,2}\s+bulan\s+\d{1,2}\s+tahun\s+\d{4})"
        r"\s+(?:membeli|beli|dibeli)\s+(.+?)\s+di\s+(.+?)\s+dengan total\w*\s+([\w\s,.]+)",
        re.IGNORECASE,
    )
    match = pattern.search(text)
    if not match:
        return None

    raw_date = match.group(1)
    items = match.group(2)
    company = match.group(3)
    raw_total = match.group(4)

    date = parse_date(raw_date)
    total = parse_total(raw_total)

    items_list = [
        item.strip()
        for item in re.split(r",\s*(?=dan)|,\s*|dan\s*", items)
        if item.strip()
    ]

    return {
        "Company": company,
        "Date": date,
        "Items": items_list,
        "Total": total,
    }

transcribed_text = result["text"]
extracted_data = extract_transcribed_data(transcribed_text)

if extracted_data:
    print("Company:", extracted_data["Company"])
    print("Date:", extracted_data["Date"])
    print("Items:")
    for item in extracted_data["Items"]:
        print(f"- {item}")
    print("Total:", extracted_data["Total"])
else:
    print("Kalimat tidak sesuai format.")


Company: toko Makmur Jaya
Date: 02-12-2024
Items:
- bakso
- soto
- ketupat
Total: 50000
